In [ ]:
import asyncio
import nest_asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from autogen_agentchat.agents import UserProxyAgent, AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
import backoff
import openai


In [8]:
# OpenAI
gpt_model = "gpt-4o-mini"

# DeepSeek Config
deepseek_model = "deepseek-reasoner"
deepseek_url = "https://api.deepseek.com"
deepseek_model_info = {
    "name": "deepseek-chat", # 模型名称，可随意填写
    "parameters": {
        "max_tokens": 2048,  # 每次输出最大token数
                             # deepseek官方数据：1个英文字符 ≈ 0.3 个 token。1 个中文字符 ≈ 0.6 个 token。
        "temperature": 0.4,  # 模型随机性参数，数字越大，生成的结果随机性越大，一般为0.7，
                             # 如果希望AI提供更多的想法，可以调大该数字
        "top_p": 0.9,  # 模型随机性参数，接近 1 时：模型几乎会考虑所有可能的词，只有概率极低的词才会被排除，随机性也越强；
                       # 接近 0 时：只有概率非常高的极少数词会被考虑，这会使模型的输出变得非常保守和确定
    },
    "family": "gpt-4o",  # 必填字段，model属于的类别
    "functions": [],  # 非必填字段，如果模型支持函数调用，可以在这里定义函数信息
    "vision": False,  # 必填字段，模型是否支持图像输入
    "json_output": True,  # 必填字段，模型是否支持json格式输出
    "function_calling": True  # 必填字段，模型是否支持函数调用，如果模型需要使用工具函数，该字段为true
}
    # model_client = OpenAIChatCompletionClient(
    #     model=deepseek_model,
    #     base_url=deepseek_url,
    #     api_key="sk-3df25786137b4387ae153c38afb00bd8", 
    #     model_info=deepseek_model_info
    # )

In [5]:
import backoff
import openai
@backoff.on_exception(backoff.expo, openai.RateLimitError)
def completions_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

completions_with_backoff(model="text-davinci-003", prompt="Once upon a time,")

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
@backoff.on_exception(backoff.expo, openai.RateLimitError)
async def example_usage() -> None:
    model_client = OpenAIChatCompletionClient(model=gpt_model)

    # Create agents
    agents = []

    user = UserProxyAgent(
        name="User", 
        description="The user agent.",
        )
    agents.append(user)
    
    surfer1 = MultimodalWebSurfer(
        name="WebSurfer1",
        model_client=model_client, 
        description="A web surfer agent that search for things to do in the destination. It also need to return the booking fee for each activity.",
        start_page="https://www.tripadvisor.com/", 
    )
    agents.append(surfer1)

    surfer2 = MultimodalWebSurfer(
        name="WebSurfer2",
        model_client=model_client, 
        description="A web surfer agent that search for accommodation in the destination. It also need to return the booking fee for each accommodation.",
        start_page="https://www.tripadvisor.com/", 
    )
    agents.append(surfer2)

    surfer3 = MultimodalWebSurfer(
        name="WebSurfer3",
        model_client=model_client, 
        description="A web surfer agent that search for flight tickets(or other travel tools) in the destination. It also need to return the booking fee for each flight(or other travel methods).",
        start_page="https://www.tripadvisor.com/",
    )
    agents.append(surfer3)

    assistant = AssistantAgent(
        name="Assistant",
        model_client=model_client,
        description="A helpful assistant agent that prints out the options.",
        # tools=
    )
    agents.append(assistant)

    cond1 = TextMentionTermination("TERMINATE")
    team = MagenticOneGroupChat(agents, model_client=model_client, termination_condition=cond1)

    task="Plan a travel itinerary. " \
    "The final plan should include the following information: place of departure(from which city, country), destination, travel dates(schedule), accommodation, activities(things to do), and budget. " \
    "First, ask the user for place of departure(from which city, country). This is mendatory infomation the user need to answer, so you need to keep asking until the user answers this. " \
    "Second, ask the user for destination, travel dates(schedule) and expected budget. Ask the usr for these one by one, and give 3 recommendation options along with your ask. " \
    # "Accommodation, activities(things to do), the information you need to provide for the user. " \
    # "You need to search for the activities, accommodations and flight(or other travel tools) separately, and the calculate the budget accordingly." \
    "Finally, you need to provide a summary of the travel plan. "
    stream = team.run_stream(task=task)
    await Console(stream)
    await cond1.reset()
    # for message in stream:
    #     print("MESSAGE", message)
    

In [12]:
if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(example_usage())

---------- user ----------
Plan a travel itinerary. The final plan should include the following information: place of departure(from which city, country), destination, travel dates(schedule), accommodation, activities(things to do), and budget. First, ask the user for place of departure(from which city, country). This is mendatory infomation the user need to answer, so you need to keep asking until the user answers this. Second, ask the user for destination, travel dates(schedule) and expected budget. Ask the usr for these one by one, and give 3 recommendation options along with your ask. 
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

Plan a travel itinerary. The final plan should include the following information: place of departure(from which city, country), destination, travel dates(schedule), accommodation, activities(things to do), and budget. First, ask the user for place of departure(from which city, country). This is menda

In [ ]:
import random

from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from typing_extensions import Annotated


async def get_stock_price(ticker: str, date: Annotated[str, "Date in YYYY/MM/DD"]) -> float:
    # Returns a random stock price for demonstration purposes.
    return random.uniform(10, 200)


# Create a function tool.
stock_price_tool = FunctionTool(get_stock_price, description="Get the stock price.")

# Run the tool.
cancellation_token = CancellationToken()
result = await stock_price_tool.run_json({"ticker": "AAPL", "date": "2021/01/01"}, cancellation_token)

# Print the result.
print(stock_price_tool.return_value_as_string(result))

In [15]:
print(st)

None
